In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import sqlite3
import numpy as np
from scipy.signal import butter, filtfilt

# Configuração Visual
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = [14, 16] # Altura maior para 4 gráficos
pd.set_option('display.max_rows', None)

### 1. Seleção do Paciente
Execute a célula abaixo para ver os pacientes e escolha o **ID do Paciente**.

In [ ]:
db_path = '../backend/clinic.db'
conn = sqlite3.connect(db_path)

df_patients = pd.read_sql_query("SELECT * FROM patients", conn)
display(df_patients)

In [ ]:
# === ESCOLHA O PACIENTE ===
PATIENT_ID = 1  # <--- DIGITE O ID DO PACIENTE AQUI
# ==========================

### 2. Seleção da Sessão
Agora veja todas as sessões deste paciente e escolha o **ID da Sessão**.

In [ ]:
query_sessions = f"SELECT id, timestamp, duration_seconds, max_angle_esq, max_angle_dir FROM sessions WHERE patient_id = {PATIENT_ID}"
df_sessions = pd.read_sql_query(query_sessions, conn)
display(df_sessions)

In [ ]:
# === ESCOLHA A SESSÃO ===
SESSION_ID = 10  # <--- DIGITE O ID DA SESSÃO AQUI
# ========================

### 3. Processamento e Plotagem (Dados Brutos)
Abaixo estão os 3 gráficos com os dados originais (Raw Data).

In [ ]:
def get_session_raw_data(session_id):
    try:
        query = f"SELECT raw_data_blob, duration_seconds FROM sessions WHERE id = {session_id}"
        row = pd.read_sql_query(query, conn).iloc[0]
        raw_json = row['raw_data_blob']
        duration = row['duration_seconds']
        data = json.loads(raw_json)
        return pd.DataFrame(data), duration
    except IndexError:
        print(f"Sessão {session_id} não encontrada!")
        return None, None

df_raw, duration = get_session_raw_data(SESSION_ID)

if df_raw is not None and not df_raw.empty:
    # --- Lógica de Tempo de Alta Resolução ---
    num_samples = len(df_raw)
    if duration and duration > 0:
        time_axis = np.linspace(0, duration, num_samples)
    else:
        time_axis = df_raw.index * 0.1
    df_raw['seconds_high_res'] = time_axis

    # --- Plotagem (3 Subplots Separados) ---
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=False, figsize=(14, 14))
    
    # Cores do App (Dashboard):
    # Ângulo: Esq=#a78bfa (Lilás Claro), Dir=#7c3aed (Roxo Escuro)
    # EMG: Esq=#60a5fa (Azul Claro), Dir=#1e40af (Azul Escuro)
    # ECG: Esq=#fbbf24 (Âmbar), Dir=#ea580c (Laranja Escuro)

    # 1. Gráfico de Ângulo
    if 'ESQ_angle' in df_raw.columns: 
        ax1.plot(df_raw['seconds_high_res'], df_raw['ESQ_angle'], label='Esq (Parética)', color='#a78bfa', linestyle='--', linewidth=2)
    if 'DIR_angle' in df_raw.columns: 
        ax1.plot(df_raw['seconds_high_res'], df_raw['DIR_angle'], label='Dir (Controle)', color='#7c3aed', linewidth=2)
    ax1.set_title(f'Ângulo da Flexão do Quadril x Tempo - Sessão {SESSION_ID}', fontsize=14)
    ax1.set_ylabel('Ângulo (°)', fontsize=12)
    ax1.set_xlabel('Tempo (segundos)', fontsize=12)
    ax1.legend(loc='upper right')
    ax1.grid(True, linestyle='--', alpha=0.6)
    ax1.set_ylim(0, 180)

    # 2. Gráfico de EMG
    if 'ESQ_emg' in df_raw.columns: 
        ax2.plot(df_raw['seconds_high_res'], df_raw['ESQ_emg'], label='Esq (Parética)', color='#60a5fa', linestyle='--', linewidth=1.5)
    if 'DIR_emg' in df_raw.columns: 
        ax2.plot(df_raw['seconds_high_res'], df_raw['DIR_emg'], label='Dir (Controle)', color='#1e40af', linewidth=1.5)
    ax2.set_title(f'Sinal EMG bruto (Reto Femoral) x tempo - Sessão {SESSION_ID}', fontsize=14)
    ax2.set_ylabel('Sinal (0-4095)', fontsize=12)
    ax2.set_xlabel('Tempo (segundos)', fontsize=12)
    ax2.legend(loc='upper right')
    ax2.grid(True, linestyle='--', alpha=0.6)

    # 3. Gráfico de ECG
    if 'ESQ_ecg' in df_raw.columns: 
        ax3.plot(df_raw['seconds_high_res'], df_raw['ESQ_ecg'], label='Esq (Parética)', color='#fbbf24', linestyle='--', linewidth=1.5)
    if 'DIR_ecg' in df_raw.columns: 
        ax3.plot(df_raw['seconds_high_res'], df_raw['DIR_ecg'], label='Dir (Controle)', color='#ea580c', linewidth=1.5)
    ax3.set_title(f'Sinal ECG bruto (Isquiotibial) x tempo - Sessão {SESSION_ID}', fontsize=14)
    ax3.set_ylabel('Sinal (0-4095)', fontsize=12)
    ax3.set_xlabel('Tempo (segundos)', fontsize=12)
    ax3.legend(loc='upper right')
    ax3.grid(True, linestyle='--', alpha=0.6)

    plt.subplots_adjust(hspace=0.4) # Espaço entre gráficos
    plt.show()
    
    print(f"Total de amostras: {num_samples}")
    print(f"Duração: {duration}s")
else:
    print("Não há dados válidos nesta sessão.")

### 4. Insight Profissional: Ativação Muscular Filtrada
Abaixo, aplicamos um filtro Butterworth (Low-pass) para visualizar a envoltória de ativação de cada músculo separadamente.

In [ ]:
# Função de Filtro Butterworth
def butter_lowpass_filter(data, cutoff, fs, order=4):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

# Parâmetros do Filtro
FS = 100  # Frequência de amostragem estimada (Hz)
CUTOFF = 3 # Frequência de corte (Hz) - 3Hz para uma envoltória bem suave

if df_raw is not None and not df_raw.empty:
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(14, 10))
    
    # --- 1. Reto Femoral (EMG) - Filtrado ---
    if 'ESQ_emg' in df_raw.columns:
        try:
            filtered_esq_emg = butter_lowpass_filter(df_raw['ESQ_emg'], CUTOFF, FS)
            ax1.plot(df_raw['seconds_high_res'], filtered_esq_emg, label='Esq (Parética)', color='#60a5fa', linestyle='--', linewidth=2)
        except Exception as e: print(f"Erro EMG Esq: {e}")
    if 'DIR_emg' in df_raw.columns:
        try:
            filtered_dir_emg = butter_lowpass_filter(df_raw['DIR_emg'], CUTOFF, FS)
            ax1.plot(df_raw['seconds_high_res'], filtered_dir_emg, label='Dir (Controle)', color='#1e40af', linewidth=2)
        except Exception as e: print(f"Erro EMG Dir: {e}")
    
    ax1.set_title(f'Ativação Muscular - Reto Femoral (EMG) - Filtrado ({CUTOFF}Hz)', fontsize=14)
    ax1.set_ylabel('Amplitude (u.a.)')
    ax1.legend(loc='upper right')
    ax1.grid(True, linestyle='--', alpha=0.6)

    # --- 2. Isquiotibial (ECG) - Filtrado ---
    if 'ESQ_ecg' in df_raw.columns:
        try:
            filtered_esq_ecg = butter_lowpass_filter(df_raw['ESQ_ecg'], CUTOFF, FS)
            ax2.plot(df_raw['seconds_high_res'], filtered_esq_ecg, label='Esq (Parética)', color='#fbbf24', linestyle='--', linewidth=2)
        except Exception as e: print(f"Erro ECG Esq: {e}")
    if 'DIR_ecg' in df_raw.columns:
        try:
            filtered_dir_ecg = butter_lowpass_filter(df_raw['DIR_ecg'], CUTOFF, FS)
            ax2.plot(df_raw['seconds_high_res'], filtered_dir_ecg, label='Dir (Controle)', color='#ea580c', linewidth=2)
        except Exception as e: print(f"Erro ECG Dir: {e}")

    ax2.set_title(f'Ativação Muscular - Isquiotibial (ECG) - Filtrado ({CUTOFF}Hz)', fontsize=14)
    ax2.set_ylabel('Amplitude (u.a.)')
    ax2.set_xlabel('Tempo (s)')
    ax2.legend(loc='upper right')
    ax2.grid(True, linestyle='--', alpha=0.6)

    plt.subplots_adjust(hspace=0.3)
    plt.show()
else:
    print("Não há dados para plotar.")

### 5. Cinemática Avançada: Filtro de Kalman
Abaixo, aplicamos um **Filtro de Kalman** para suavizar o sinal de ângulo do joelho, reduzindo ruídos e flutuações.

In [ ]:
# Implementação Simples do Filtro de Kalman (1D)
def kalman_filter_1d(data, Q=1e-5, R=0.01):
    """
    Aplica um filtro de Kalman 1D simples.
    data: array de medidas (sinal ruidoso)
    Q: covariância do ruído do processo (confiança na previsão)
    R: covariância do ruído da medição (confiança na medida)
    """
    n_iter = len(data)
    sz = (n_iter,)
    
    # Alocação de espaço
    xhat = np.zeros(sz)      # Estimativa a posteriori do estado
    P = np.zeros(sz)         # Estimativa a posteriori da covariância do erro
    xhatminus = np.zeros(sz) # Estimativa a priori do estado
    Pminus = np.zeros(sz)    # Estimativa a priori da covariância do erro
    K = np.zeros(sz)         # Ganho de Kalman

    # Valores iniciais
    xhat[0] = data[0]
    P[0] = 1.0

    for k in range(1, n_iter):
        # 1. Predição (Time Update)
        xhatminus[k] = xhat[k-1]
        Pminus[k] = P[k-1] + Q

        # 2. Correção (Measurement Update)
        K[k] = Pminus[k] / (Pminus[k] + R)
        xhat[k] = xhatminus[k] + K[k] * (data[k] - xhatminus[k])
        P[k] = (1 - K[k]) * Pminus[k]

    return xhat

if df_raw is not None and not df_raw.empty:
    plt.figure(figsize=(14, 6))

    # --- Processamento Perna DIREITA (Controle) ---
    if 'DIR_angle' in df_raw.columns:
        kalman_dir = kalman_filter_1d(df_raw['DIR_angle'].values)
        # Plot Kalman (frente)
        plt.plot(df_raw['seconds_high_res'], kalman_dir, color='#7c3aed', linewidth=2.5, label='Dir (Kalman)')

    # --- Processamento Perna ESQUERDA (Parética) ---
    if 'ESQ_angle' in df_raw.columns:
        kalman_esq = kalman_filter_1d(df_raw['ESQ_angle'].values)
        # Plot Kalman (frente)
        plt.plot(df_raw['seconds_high_res'], kalman_esq, color='#a78bfa', linewidth=2.5, linestyle='--', label='Esq (Kalman)')

    plt.title('Cinemática Avançada: Ângulo da Flexão do Quadril - Filtro de Kalman', fontsize=14)
    plt.xlabel('Tempo (s)')
    plt.ylabel('Ângulo (°)')
    plt.legend(loc='upper right')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.ylim(0, 180)
    plt.show()
else:
    print("Não há dados para plotar.")